In [1]:
import serial
import time

class ElliptecController:
    def __init__(self, port='COM3', baudrate=9600, address='0'):
        self.port = port
        self.baudrate = baudrate
        self.address = address  # hex address (0‑F)
        self.ser = serial.Serial(port=self.port,
                                 baudrate=self.baudrate,
                                 bytesize=serial.EIGHTBITS,
                                 stopbits=serial.STOPBITS_ONE,
                                 parity=serial.PARITY_NONE,
                                 timeout=2)  # according to manual, timeout 2 s
        time.sleep(0.1)
        self._purge()

    def _purge(self):
        self.ser.reset_input_buffer()
        self.ser.reset_output_buffer()
        time.sleep(0.05)

    def _send_cmd(self, cmd: str):
        """Send command (without CRLF), add CR (0x0D) and LF (0x0A) terminator according to manual"""
        full = f"{self.address}{cmd}"
        # send as ASCII bytes
        self.ser.write(full.encode('ascii'))
        self.ser.write(b'\r\n')
        # wait for response
        resp = self.ser.readline().decode('ascii', errors='ignore').strip()
        return resp

    def get_info(self):
        """Command IN: get module information"""
        resp = self._send_cmd("in")
        return resp

    def get_status(self):
        """Command GS: get status/error"""
        resp = self._send_cmd("gs")
        return resp

    def home(self, direction=0):
        """Command HO: homing. direction=0 for CW / forward (rotary) or ignored if linear"""
        cmd = f"ho{direction}"
        resp = self._send_cmd(cmd)
        return resp

    def move_absolute(self, pulses: int):
        """Command MA: move to absolute position in pulse counts"""
        # pulses must be encoded as 32-bit (4 bytes) big endian hexadecimal
        hexval = format(pulses & 0xFFFFFFFF, '08X')
        cmd = f"ma{hexval}"
        resp = self._send_cmd(cmd)
        return resp

    def move_relative(self, pulses: int):
        """Command MR: move relative by pulse counts"""
        hexval = format(pulses & 0xFFFFFFFF, '08X')
        cmd = f"mr{hexval}"
        resp = self._send_cmd(cmd)
        return resp

    def stop(self):
        """Command ST: stop motion"""
        resp = self._send_cmd("st")
        return resp

    def close(self):
        self.ser.close()

    def __enter__(self):
        return self

    def __exit__(self, exc_type, exc_val, exc_tb):
        self.close()

In [6]:
# Mount configuration: number of pulses per turn
PULSES_PER_TURN = 65536  # adjust according to your module

def deg_to_pulses(degrees):
    """Convert degrees to pulses (integer)."""
    return int(degrees * PULSES_PER_TURN / 360)

def pulses_to_deg(pulses):
    """Convert pulses to degrees (float)."""
    # Handle signed 32-bit values
    if pulses >= 0x80000000:
        pulses -= 0x100000000
    return pulses * 360.0 / PULSES_PER_TURN

def get_position_degrees(ctl):
    """Read current position and convert to degrees."""
    resp = ctl._send_cmd("gs")  # GS returns status only; to read actual position, use a dummy move of 0 pulses or the PO response after a move
    # For demo purposes, assume last PO response is returned from previous command
    # In practice, you may need a proper get_position() method in your class
    return resp  # Replace with actual conversion if PO value is available

# Open COM3 once
ctl = ElliptecController(port='COM3', address='0')

# ===== Information =====
ctl.address = '0'
print("=== Device 0 Info ===", ctl.get_info())
print("=== Device 0 Status ===", ctl.get_status())

ctl.address = '2'
print("=== Device 2 Info ===", ctl.get_info())
print("=== Device 2 Status ===", ctl.get_status())

# ===== Homing =====
ctl.address = '0'
resp0 = ctl.home(direction=0) # Homing device 0
ctl.address = '2'
resp1 = ctl.home(direction=0) # Homing device 1
time.sleep(1)

# ===== Move Absolute (degrees) =====
ctl.address = '0'
resp0 = ctl.move_absolute(deg_to_pulses(10)) # Rotate device 0 to an absolute position
ctl.address = '2'
resp1 = ctl.move_absolute(deg_to_pulses(135)) # Rotate device 1 to an absolute position

# ===== Move Relative (degrees) =====
ctl.address = '0'
resp0 = ctl.move_relative(deg_to_pulses(45)) # Rotate device 0 clockwise relatively to the last position
ctl.address = '2'
resp1 = ctl.move_relative(deg_to_pulses(60)) # Rotate device 1 clockwise relatively to the last position

ctl.address = '0'
resp0 = ctl.move_relative(deg_to_pulses(-27)) # Rotate device 0 anticlockwise relatively to the last position
ctl.address = '2'
resp1 = ctl.move_relative(deg_to_pulses(-36)) # Rotate device 1 anticlockwise relatively to the last position

# ===== Read Position (degrees) =====
print("Device 0 Position (deg) …", round(pulses_to_deg(int(resp0[-8:], 16)), 2))
print("Device 1 Position (deg) …", round(pulses_to_deg(int(resp1[-8:], 16)), 2))

# ===== Stop =====
ctl.address = '0'
print("Stop Device 0 …", ctl.stop())

ctl.address = '2'
print("Stop Device 2 …", ctl.stop())

# Close COM port
ctl.close()

=== Device 0 Info === 0IN0E1140132020251801016800023000
=== Device 0 Status === 0GS00
=== Device 2 Info === 2IN0E1140134020251801016800023000
=== Device 2 Status === 2GS00
Device 0 Position (deg) … 28.01
Device 1 Position (deg) … 158.99
Stop Device 0 … 0GS00
Stop Device 2 … 2GS00


In [ ]:
import serial

try:
    ser = serial.Serial('COM3', 9600, timeout=1)
    print("COM3 opened successfully")
    ser.close()
except serial.SerialException as e:
    print("Cannot open COM3:", e)

Cannot open COM3: could not open port 'COM3': PermissionError(13, 'Zugriff verweigert', None, 5)
